### Import necessary packages

In [1]:
import pandas as pd
import numpy as np
import csv
import glob

Due to the large filesize of the datasets, the data cannot be pushed to Github. The datasets and final combined dataset is available on this Google Drive link here.

In [2]:
filenames = glob.glob('data/raw/*.csv') 

#The column names change after a certain point, have to find when the columns change

for file in filenames:
    with open(file, newline = '') as f:
        reader = csv.reader(f)
        columns = next(reader)
        print(file, columns, '\n')

data\2010-capitalbikeshare-tripdata.csv ['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type'] 

data\2011-capitalbikeshare-tripdata.csv ['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type'] 

data\2012Q1-capitalbikeshare-tripdata.csv ['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type'] 

data\2012Q2-capitalbikeshare-tripdata.csv ['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type'] 

data\2012Q3-capitalbikeshare-tripdata.csv ['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type'] 

data\2012Q4-capitalbikeshare-tripdata.csv ['Durat

Column names change starting with `202003-capitalbikeshare-tripdata.csv`. To merge all these files into one with same column names, they will first be loaded into two different dataframes and columns will be dropped/renamed so that they can both be merged into one final dataframe.

For datasets `2010-capitalbikeshare-tripdata.csv` to `202003-capitalbikeshare-tripdata.csv` the following changes will be made:
* `Start Date` -> `started_at`
* `End date` -> `ended_at`
* `Start station` -> `start_station_name`
* `Start station number` -> `start_station_id`
* `End station` -> `end_station_name`
* `End station number` -> `end_station_id`
* `Bike number` dropped (doesn't exist in recent datasets)
* `Member type` -> `member_casual`

For datasets `202004-capitalbikeshare-tripdata.csv` and onwards, the following changes will be made:
* `ride_id` -> dropped
* `start_station_id` -> dropped
* `end_station_id` -> dropped

### Import datasets into two separate dataframes

In [3]:
filenames[:-8]

dfs = []

cols = ['duration', 'started_at', 'ended_at', 'start_station_id', 'start_station_name', 'end_station_id', 'end_station_name', 'bike_number', 'member_casual']

for file in filenames[:-8]:
    tmp = pd.read_csv(file, index_col = None, names = cols, header = 0)
    dfs.append(tmp)
    
df1 = pd.concat(dfs, axis=0, ignore_index = True)

df1.head()

,duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,bike_number,member_casual
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,M St & New Jersey Ave SE,31108,4th & M St SW,W00742,Member
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,1st & N St SE,31209,1st & N St SE,W00032,Member
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,5th & K St NW,31100,19th St & Pennsylvania Ave NW,W00993,Member
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,5th & K St NW,31602,Park Rd & Holmead Pl NW,W00344,Member
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,19th St & Pennsylvania Ave NW,31201,15th & P St NW,W00883,Member


In [4]:
df1.shape

(26596144, 9)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26596144 entries, 0 to 26596143
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   duration            int64 
 1   started_at          object
 2   ended_at            object
 3   start_station_id    int64 
 4   start_station_name  object
 5   end_station_id      int64 
 6   end_station_name    object
 7   bike_number         object
 8   member_casual       object
dtypes: int64(3), object(6)
memory usage: 1.8+ GB


In [6]:
df1.isna().sum() #Thankfully only 14 null values in 'bike_number', which will be dropped anyway

duration               0
started_at             0
ended_at               0
start_station_id       0
start_station_name     0
end_station_id         0
end_station_name       0
bike_number           14
member_casual          0
dtype: int64

In [7]:
df1.drop(['bike_number', 'start_station_id', 'end_station_id'], axis = 1, inplace = True)

In [8]:
dfs = []
for file in filenames[-8:]:
    tmp = pd.read_csv(file, header = 0, index_col = None, )
    dfs.append(tmp)
    
df2 = pd.concat(dfs, axis=0, ignore_index = True)

In [9]:
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,is_equity
0,495C10604A58ACA9,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.8767,-77.0178,38.8990,-77.0334,member,NaN
1,08A5138C88D182A5,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,100.0,4th & M St SW,21.0,38.9028,-77.0386,38.8767,-77.0178,member,NaN
2,505E308BDCA9C792,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,578.0,4th & M St SW,21.0,38.8769,-77.0061,38.8767,-77.0178,member,NaN
3,D3436FC6CCF04110,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.9295,-77.0278,38.9059,-77.0325,casual,NaN
4,37B8457C89257BD5,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.9004,-76.9883,38.8870,-76.9968,member,NaN


In [10]:
# There is a column 'is_equity' that is not listed or described on the official Capital BikeShare website
# This column will be removed.
df2['is_equity'].unique()

array([nan, False, True], dtype=object)

In [11]:
df2.drop(['ride_id', 'is_equity'], axis = 1, inplace = True)
df2.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.8767,-77.0178,38.8990,-77.0334,member
1,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,100.0,4th & M St SW,21.0,38.9028,-77.0386,38.8767,-77.0178,member
2,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,578.0,4th & M St SW,21.0,38.8769,-77.0061,38.8767,-77.0178,member
3,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.9295,-77.0278,38.9059,-77.0325,casual
4,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.9004,-76.9883,38.8870,-76.9968,member


In [12]:
df2.shape

(1573532, 12)

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573532 entries, 0 to 1573531
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   rideable_type       1573532 non-null  object 
 1   started_at          1573532 non-null  object 
 2   ended_at            1573532 non-null  object 
 3   start_station_name  1499953 non-null  object 
 4   start_station_id    1499953 non-null  float64
 5   end_station_name    1488020 non-null  object 
 6   end_station_id      1488020 non-null  float64
 7   start_lat           1573524 non-null  float64
 8   start_lng           1573524 non-null  float64
 9   end_lat             1566991 non-null  float64
 10  end_lng             1566991 non-null  float64
 11  member_casual       1573532 non-null  object 
dtypes: float64(6), object(6)
memory usage: 144.1+ MB


### Take care of null values

In [14]:
df2.isna().sum()

rideable_type             0
started_at                0
ended_at                  0
start_station_name    73579
start_station_id      73579
end_station_name      85512
end_station_id        85512
start_lat                 8
start_lng                 8
end_lat                6541
end_lng                6541
member_casual             0
dtype: int64

There's definitely a lot more null values in this dataset. Time to investigate what the issue might be.

In [15]:
df2[df2['start_lat'].isnull() & df2['start_lng'].isnull()]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
655851,docked_bike,2020-07-29 11:19:11,2020-07-29 11:23:21,Motivate Tech Office,285.0,Motivate BX Tech office,283.0,NaN,NaN,38.964406,-77.010759,member
853971,docked_bike,2020-08-12 11:40:29,2020-08-12 11:40:36,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
853972,docked_bike,2020-08-12 12:17:04,2020-08-12 12:17:08,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
853973,docked_bike,2020-08-12 12:02:22,2020-08-12 12:02:26,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
853974,docked_bike,2020-08-12 12:28:08,2020-08-12 12:28:52,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
853975,docked_bike,2020-08-12 11:45:01,2020-08-12 11:45:06,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
853976,docked_bike,2020-08-12 11:51:55,2020-08-12 11:51:58,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member
855654,docked_bike,2020-08-10 13:18:09,2020-08-10 13:18:14,Motivate Tech Office,285.0,Motivate Tech Office,285.0,NaN,NaN,NaN,NaN,member


There were only 8 instances where the start station latitude and longitude are missing. These entries had the name of the start station 'Motivate Tech Office', however when I tried to manually look up this location on Google Maps, nothing came up.

In [16]:
df2[df2['end_lat'].isnull() & df2['end_lng'].isnull()]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
90,docked_bike,2020-04-29 18:14:48,2020-04-29 18:25:28,13th & D St NE,146.0,NaN,NaN,38.894800,-76.987600,NaN,NaN,casual
283,docked_bike,2020-04-10 16:57:46,2020-04-10 17:19:06,21st St & Constitution Ave NW,173.0,NaN,NaN,38.892500,-77.046600,NaN,NaN,member
737,docked_bike,2020-04-04 12:37:19,2020-04-04 16:27:39,Market Square / King St & Royal St,238.0,NaN,NaN,38.804700,-77.043400,NaN,NaN,casual
1715,docked_bike,2020-04-12 10:25:54,2020-04-13 11:25:48,8th & O St NW,413.0,NaN,NaN,38.908600,-77.022800,NaN,NaN,casual
1739,docked_bike,2020-04-11 13:02:39,2020-04-12 13:24:25,14th & Belmont St NW,81.0,NaN,NaN,38.921100,-77.031900,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...
1557652,docked_bike,2020-11-01 13:56:08,2020-11-02 11:23:01,20th & L St NW,266.0,NaN,NaN,38.903584,-77.044789,NaN,NaN,member
1558002,docked_bike,2020-11-10 16:06:07,2020-11-10 16:11:52,Clarendon Blvd & N Fillmore St,176.0,NaN,NaN,38.888100,-77.093080,NaN,NaN,member
1562902,docked_bike,2020-11-12 20:23:35,2020-11-12 20:34:14,14th & Rhode Island Ave NW,27.0,NaN,NaN,38.908600,-77.032300,NaN,NaN,casual
1563012,docked_bike,2020-11-07 13:14:22,2020-11-07 14:13:55,New Hampshire Ave & 24th St NW,382.0,NaN,NaN,38.901755,-77.051084,NaN,NaN,casual


Looks like a lot of the entries with missing end latitude and longitude do not have any station names or ids. Let's see if that's really the case. If they do have names or ID's, the information could be used to fill the latitude and longitudes.

In [17]:
df2[df2['end_lat'].isnull() & df2['end_lng'].isnull()]['end_station_id'].unique()

array([ nan, 285.])

In [18]:
df2[df2['end_lat'].isnull() & df2['end_lng'].isnull()]['end_station_name'].unique()

array([nan, 'Motivate Tech Office'], dtype=object)

It turns out that the only instances where the latitude and longitude are missing have no usable station names or ID's. These entries might be cases where people rented the bikes but never properly returned them and thus an ending station ID,name, or latitude/longitude were never recorded. We can't be entirely sure what the cause may be, therefore these entries will just be dropped.

In [19]:
df2.dropna(subset = ['start_lat', 'start_lng', 'end_lat', 'end_lng'], inplace = True)
df2.isna().sum()

rideable_type             0
started_at                0
ended_at                  0
start_station_name    73579
start_station_id      73579
end_station_name      78978
end_station_id        78978
start_lat                 0
start_lng                 0
end_lat                   0
end_lng                   0
member_casual             0
dtype: int64

In [20]:
df2[df2['start_station_name'].isnull()]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
244240,electric_bike,2020-06-29 15:09:52,2020-06-29 15:13:38,NaN,NaN,NaN,NaN,38.97,-77.02,38.97,-77.02,member
244241,electric_bike,2020-06-29 15:18:15,2020-06-29 15:26:04,NaN,NaN,NaN,NaN,38.97,-77.02,38.96,-77.01,member
244242,electric_bike,2020-06-29 15:03:12,2020-06-29 15:06:51,NaN,NaN,NaN,NaN,38.96,-77.01,38.97,-77.02,member
244243,electric_bike,2020-06-17 15:33:59,2020-06-17 15:34:55,NaN,NaN,NaN,NaN,38.96,-77.01,38.96,-77.01,member
244244,electric_bike,2020-06-17 14:53:21,2020-06-17 14:58:11,NaN,NaN,NaN,NaN,38.96,-77.01,38.96,-77.01,member
...,...,...,...,...,...,...,...,...,...,...,...,...
1573510,electric_bike,2020-11-24 11:56:43,2020-11-24 12:26:41,NaN,NaN,NaN,NaN,38.93,-77.03,38.93,-77.03,casual
1573527,electric_bike,2020-11-10 11:10:51,2020-11-10 11:55:07,NaN,NaN,NaN,NaN,38.91,-77.00,38.91,-77.01,casual
1573528,electric_bike,2020-11-10 13:40:56,2020-11-10 14:10:00,NaN,NaN,NaN,NaN,38.92,-77.01,39.00,-77.03,casual
1573530,electric_bike,2020-11-10 15:05:42,2020-11-10 15:09:21,NaN,NaN,NaN,NaN,38.95,-77.08,38.95,-77.08,casual


So although the `start_station_name` and `start_station_id` aren't listed, the latitude and longitude exist so maybe the information can be extracted from other entries with similar latitude and longitude. Also, it seems like maybe it has something to do with these entries being electric_bike rentals. 

In [21]:
df2[df2['start_station_name'].isnull()]['rideable_type'].unique()

array(['electric_bike'], dtype=object)

Yes, all of the entries without `start_station_name` have to do with electric bike rentals.

In [22]:
len(df2[df2['rideable_type'] == 'electric_bike'])

210809

But not all `electric_bike` rentals have this issue.

In [23]:
# Starting/ending lat long will be merged into one column for easier manipulation

df2['start_lat_lng'] = list(zip(df2.start_lat, df2.start_lng))
df2['end_lat_lng'] = list(zip(df2.end_lat, df2.end_lng))
df2.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_lat_lng,end_lat_lng
0,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.8767,-77.0178,38.8990,-77.0334,member,"(38.8767, -77.0178)","(38.899, -77.0334)"
1,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,100.0,4th & M St SW,21.0,38.9028,-77.0386,38.8767,-77.0178,member,"(38.9028, -77.0386)","(38.8767, -77.0178)"
2,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,578.0,4th & M St SW,21.0,38.8769,-77.0061,38.8767,-77.0178,member,"(38.8769, -77.0061)","(38.8767, -77.0178)"
3,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.9295,-77.0278,38.9059,-77.0325,casual,"(38.9295, -77.0278)","(38.9059, -77.0325)"
4,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.9004,-76.9883,38.8870,-76.9968,member,"(38.9004, -76.9883)","(38.887, -76.9968)"


In [24]:
null_start = df2[df2['start_station_name'].isnull()]
null_station_coords = list(null_start.start_lat_lng.unique())
len(null_station_coords)

543

In [25]:
null_end = df2[df2['end_station_name'].isnull()]
null_end_coords = list(null_end.end_lat_lng.unique())
len(null_end_coords)

684

In [26]:
null_station_coords.extend(null_end_coords)
len(null_station_coords)

1227

In [27]:
def removeDuplicates(lst):
    return [t for t in (set(tuple(i) for i in lst))]

null_station_coords = removeDuplicates(null_station_coords)
len(null_station_coords)

686

In [28]:
for coord in null_station_coords:
    address_check = df2[df2['start_lat_lng']==coord]['start_station_name'].unique()
    if len(address_check) > 1:
        # 'address_check' will only return a list of ['nan'], check if there are other names for station other than 'nan'
        print(f'{coord}: {address_check}')

The coordinates where `start_station` is `NaN` have no station names at all in the dataset. Could obtain street names for these coordinates.

### Obtain street names for null station entries

In [29]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent = 'test_locator')
location = geolocator.reverse(null_station_coords[0])

location.raw # can extract road from here and fill in the station

{'place_id': 275990459,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 6115499,
 'lat': '38.819905',
 'lon': '-76.989901',
 'display_name': "1100, Kennebec Street, Glassmanor, Prince George's County, Maryland, 20745, United States",
 'address': {'house_number': '1100',
  'road': 'Kennebec Street',
  'town': 'Glassmanor',
  'county': "Prince George's County",
  'state': 'Maryland',
  'postcode': '20745',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['38.819855', '38.819955', '-76.989951', '-76.989851']}

In [30]:
streets_to_fill = dict()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 0.5)
i = 0
for coord in null_station_coords:
    location = geolocator.reverse(coord)
    address = location.raw['address']
    
    if 'road' in address:
        street = address['road']
    elif 'residential' in address:
        street = address['residential']
    elif 'leisure' in address:
        street = address['leisure']
    elif 'quarter' in address:
        street = address['quarter']
    else:
        print(coord, address)
        street = list(address.values())[0]

    
    streets_to_fill[coord] = street
    

(38.89, -77.04) {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
(38.79, -76.98) {'hamlet': 'Rosecroft Park', 'town': 'Oxon Hill', 'county': "Prince George's County", 'state': 'Maryland', 'postcode': '20744-3008', 'country': 'United States', 'country_code': 'us'}
(39.06, -77.09) {'neighbourhood': 'Stoney Brook', 'hamlet': 'Franklin Park', 'county': 'Montgomery County', 'state': 'Maryland', 'postcode': '20857', 'country': 'United States', 'country_code': 'us'}
(38.89, -76.88) {'neighbourhood': 'Yorkshire Knolls', 'suburb': 'Carmody Hills', 'village': 'Peppermill Village', 'county': "Prince George's County", 'state': 'Maryland', 'postcode': '20743', 'country': 'United States', 'country_code': 'us'}
(38.96, -76.88) {'town': 'New Carrollton', 'county': "Prince George's County", 'state': 'Maryland', 'postcode': '20784', 'country': 'United States', 'country_code': 'us'}
(38.86, -77.01) {'city': 'Washington', 'state': 'District of Colum

In [31]:
# Fill start station name if empty using coordinates and dictionary created above.
df2['start_station_name'] = df2.apply(
    lambda row: streets_to_fill[row['start_lat_lng']] if row['start_station_name'] is np.nan else row['start_station_name'],
    axis = 1)

In [32]:
df2['end_station_name'] = df2.apply(
    lambda row: streets_to_fill[row['end_lat_lng']] if row['end_station_name'] is np.nan else row['end_station_name'],
    axis = 1)

In [33]:
df2.isna().sum()

rideable_type             0
started_at                0
ended_at                  0
start_station_name        0
start_station_id      73579
end_station_name          0
end_station_id        78978
start_lat                 0
start_lng                 0
end_lat                   0
end_lng                   0
member_casual             0
start_lat_lng             0
end_lat_lng               0
dtype: int64

In [34]:
df2.drop(['start_station_id', 'end_station_id'], axis = 1, inplace = True)

In [35]:
df1.head()

,duration,started_at,ended_at,start_station_name,end_station_name,member_casual
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,M St & New Jersey Ave SE,4th & M St SW,Member
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,1st & N St SE,1st & N St SE,Member
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,5th & K St NW,19th St & Pennsylvania Ave NW,Member
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,5th & K St NW,Park Rd & Holmead Pl NW,Member
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,19th St & Pennsylvania Ave NW,15th & P St NW,Member


In [36]:
df2.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,start_lat_lng,end_lat_lng
0,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,New York Ave & 15th St NW,38.8767,-77.0178,38.8990,-77.0334,member,"(38.8767, -77.0178)","(38.899, -77.0334)"
1,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,4th & M St SW,38.9028,-77.0386,38.8767,-77.0178,member,"(38.9028, -77.0386)","(38.8767, -77.0178)"
2,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,4th & M St SW,38.8769,-77.0061,38.8767,-77.0178,member,"(38.8769, -77.0061)","(38.8767, -77.0178)"
3,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,Thomas Circle,38.9295,-77.0278,38.9059,-77.0325,casual,"(38.9295, -77.0278)","(38.9059, -77.0325)"
4,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,Eastern Market / 7th & North Carolina Ave SE,38.9004,-76.9883,38.8870,-76.9968,member,"(38.9004, -76.9883)","(38.887, -76.9968)"


### Calculate trip durations for second dataframe

In [37]:
df2['started_at'] = pd.to_datetime(df2['started_at'])
df2['ended_at'] = pd.to_datetime(df2['ended_at'])

In [38]:
import datetime as dt
df2['duration'] = (df2['ended_at'] - df2['started_at']).dt.total_seconds()

In [39]:
df2.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,start_lat_lng,end_lat_lng,duration
0,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,New York Ave & 15th St NW,38.8767,-77.0178,38.8990,-77.0334,member,"(38.8767, -77.0178)","(38.899, -77.0334)",1218.0
1,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,4th & M St SW,38.9028,-77.0386,38.8767,-77.0178,member,"(38.9028, -77.0386)","(38.8767, -77.0178)",1368.0
2,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,4th & M St SW,38.8769,-77.0061,38.8767,-77.0178,member,"(38.8769, -77.0061)","(38.8767, -77.0178)",527.0
3,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,Thomas Circle,38.9295,-77.0278,38.9059,-77.0325,casual,"(38.9295, -77.0278)","(38.9059, -77.0325)",725.0
4,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,Eastern Market / 7th & North Carolina Ave SE,38.9004,-76.9883,38.8870,-76.9968,member,"(38.9004, -76.9883)","(38.887, -76.9968)",527.0


### Fill in latitude/longitude in first dataframe

In [64]:
stations = list(df2['start_station_name'])
stations.extend(df2['end_station_name'])
stations = list(set(stations))

len(stations)

1227

In [41]:
stations_dict = {}
start_stations = list(df2['start_station_name'].unique())
end_stations = list(df2['end_station_name'].unique())

for station in stations:
    if station in start_stations:
        stations_dict[station] = tuple(df2.loc[(df2['start_station_name'] == station).idxmax(), ['start_lat', 'start_lng']])
    else:
        stations_dict[station] = tuple(df2.loc[(df2['end_station_name'] == station).idxmax(), ['end_lat', 'end_lng']])

In [42]:
def fill_latlng(address):
    if address in stations_dict:
        lat, lng = stations_dict[address]
        return lat, lng
    else:
        pass
    

In [43]:
df1['start_lat_lng'] = df1['start_station_name'].apply(fill_latlng)
df1['end_lat_lng'] = df1['end_station_name'].apply(fill_latlng)

In [59]:
df1[['start_lat', 'start_lng']] = pd.DataFrame(df1['start_lat_lng'].tolist())
df1[['end_lat', 'end_lng']] = pd.DataFrame(df1['end_lat_lng'].tolist())
df1.head()

,duration,started_at,ended_at,start_station_name,end_station_name,member_casual,start_lat_lng,end_lat_lng,start_lat,start_lng,end_lat,end_lng
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,M St & New Jersey Ave SE,4th & M St SW,Member,"(38.8763, -77.0037)","(38.8767, -77.0178)",38.8763,-77.0037,38.8767,-77.0178
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,1st & N St SE,1st & N St SE,Member,"(38.8743, -77.0057)","(38.8743, -77.0057)",38.8743,-77.0057,38.8743,-77.0057
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,5th & K St NW,19th St & Pennsylvania Ave NW,Member,"(38.903, -77.01899999999999)","(38.9003, -77.0429)",38.9030,-77.0190,38.9003,-77.0429
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,5th & K St NW,Park Rd & Holmead Pl NW,Member,"(38.903, -77.01899999999999)","(38.9308, -77.0315)",38.9030,-77.0190,38.9308,-77.0315
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,19th St & Pennsylvania Ave NW,15th & P St NW,Member,"(38.9003, -77.0429)","(38.9099, -77.0344)",38.9003,-77.0429,38.9099,-77.0344


### Merge dataframes and export final dataframe

In [60]:
final_df = pd.concat([df1,df2], axis = 0, ignore_index = True)

In [45]:
final_df.head()

,duration,started_at,ended_at,start_station_name,end_station_name,member_casual,start_lat_lng,end_lat_lng,rideable_type,start_lat,start_lng,end_lat,end_lng
0,1012.0,2010-09-20 11:27:04,2010-09-20 11:43:56,M St & New Jersey Ave SE,4th & M St SW,Member,"(38.8763, -77.0037)","(38.8767, -77.0178)",NaN,NaN,NaN,NaN,NaN
1,61.0,2010-09-20 11:41:22,2010-09-20 11:42:23,1st & N St SE,1st & N St SE,Member,"(38.8743, -77.0057)","(38.8743, -77.0057)",NaN,NaN,NaN,NaN,NaN
2,2690.0,2010-09-20 12:05:37,2010-09-20 12:50:27,5th & K St NW,19th St & Pennsylvania Ave NW,Member,"(38.903, -77.01899999999999)","(38.9003, -77.0429)",NaN,NaN,NaN,NaN,NaN
3,1406.0,2010-09-20 12:06:05,2010-09-20 12:29:32,5th & K St NW,Park Rd & Holmead Pl NW,Member,"(38.903, -77.01899999999999)","(38.9308, -77.0315)",NaN,NaN,NaN,NaN,NaN
4,1413.0,2010-09-20 12:10:43,2010-09-20 12:34:17,19th St & Pennsylvania Ave NW,15th & P St NW,Member,"(38.9003, -77.0429)","(38.9099, -77.0344)",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28163129,2656.0,2020-11-10 11:10:51,2020-11-10 11:55:07,New York Avenue Northeast,P Street Northwest,casual,"(38.91, -77.0)","(38.91, -77.01)",electric_bike,38.910000,-77.000000,38.91,-77.01
28163130,1744.0,2020-11-10 13:40:56,2020-11-10 14:10:00,Adams Street Northwest,Georgia Avenue,casual,"(38.92, -77.01)","(39.0, -77.03)",electric_bike,38.920000,-77.010000,39.00,-77.03
28163131,541.0,2020-11-10 13:37:16,2020-11-10 13:46:17,Wisconsin Ave & Newark St NW,41st Street Northwest,casual,"(38.934946999999994, -77.072849)","(38.95, -77.08)",electric_bike,38.934947,-77.072849,38.95,-77.08
28163132,219.0,2020-11-10 15:05:42,2020-11-10 15:09:21,41st Street Northwest,41st Street Northwest,casual,"(38.95, -77.08)","(38.95, -77.08)",electric_bike,38.950000,-77.080000,38.95,-77.08


In [67]:
# final dataset is too big (5GB) for one csv, will be split into multiple
for id, df_i in  enumerate(np.array_split(final_df, 5)):
    df_i.to_csv(f'data/combined_data{id}.csv')